In [102]:
import numpy as np
import pandas as pd
import pickle
import tqdm.auto as tqdm
import re

In [103]:
def load_csvs(pathname):
    RESULTS_FULL_PATH_CSV = "../results/" + pathname + "_full.csv"
    RESULTS_PREREQ_PATH_CSV = "../results/" + pathname + "_prereq.csv"
    COEFS_FULL_PATH_CSV = "../results/" + pathname + "_coefs_full.csv"
    COEFS_PREREQ_PATH_CSV = "../results/" + pathname + "_coefs_prereq.csv"
    
    results_full_df = pd.read_csv(RESULTS_FULL_PATH_CSV, low_memory=False).rename(
        columns={"Unnamed: 0":"course"}).set_index("course")
    results_prereq_df = pd.read_csv(RESULTS_PREREQ_PATH_CSV, low_memory=False).rename(
        columns={"Unnamed: 0":"course"}).set_index("course")
    coefs_full_df = pd.read_csv(COEFS_FULL_PATH_CSV, low_memory=False).rename(
        columns={"Unnamed: 0":"course"}).set_index("course")
    coefs_prereq_df = pd.read_csv(COEFS_PREREQ_PATH_CSV, low_memory=False).rename(
        columns={"Unnamed: 0":"course"}).set_index("course")
    
    return results_full_df, results_prereq_df, coefs_full_df, coefs_prereq_df

# modifies the df in place
def remove_advcourse_coefs(df):
    RE_PATTERN = r"(.+?)(\d+)"

    for ind in df.index:
        ind_split = re.split(RE_PATTERN, ind)
        ind_dept = ind_split[1]
        ind_num = int(ind_split[2])
        for col in highach_coefs_full.columns:
            col_split = re.split(RE_PATTERN, col)
            col_dept = col_split[1]
            col_num = int(col_split[2])

            # same department, course code's hundreds digit is higher
            if ind_dept == col_dept and col_num // 100 > ind_num // 100:
                df.loc[ind, col] = None
                
def rowwise_normalize(df):
    return df.div(df.sum(axis=1), axis=0)

### high-achievement

In [104]:
pathname = "high-ach"

highach_results_full, highach_results_prereq, highach_coefs_full, highach_coefs_prereq = load_csvs(pathname)
display(highach_coefs_full)

,CS161,CS109,MATH51,ECON1,CS103,PSYCH50,CS224U,CS221,CS145,CS148,...,PHYSICS364,COMM122,ME300B,MUSIC421A,COMM274D,GEOPHYS281,LINGUIST205A,PHIL180A,PHIL350A,MATH248
course,,,,,,,,,,,,,,,,,,,,,
CS276,0.038613,0.037024,0.030470,0.021500,0.019509,0.018370,0.018101,0.016456,0.014715,0.013437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CEE176B,0.006059,0.000000,0.016956,0.012128,0.000000,0.000000,0.000000,-0.003740,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MS&E252,0.016742,0.054770,0.051094,0.021722,0.066643,0.015836,0.000000,0.013272,0.017568,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EARTHSYS111,0.000000,0.000000,0.018820,0.080774,0.010652,0.000806,NaN,0.000000,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PSYCH131,-0.004587,0.005964,0.021081,0.005305,0.007391,-0.008478,0.000708,-0.004582,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHEMENG162,NaN,0.003456,0.004683,0.005080,0.003456,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ME140,NaN,0.009609,0.032296,0.015827,-0.002503,NaN,NaN,0.002054,0.000000,0.003095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HUMBIO128,NaN,NaN,0.009460,0.000000,0.001916,0.001954,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
# remove coefs for courses in the same dept that are higher level (e.g. CS221 shouldn't be a predictor for CS106A)
remove_advcourse_coefs(highach_coefs_full)
display(highach_coefs_full)

,CS161,CS109,MATH51,ECON1,CS103,PSYCH50,CS224U,CS221,CS145,CS148,...,PHYSICS364,COMM122,ME300B,MUSIC421A,COMM274D,GEOPHYS281,LINGUIST205A,PHIL180A,PHIL350A,MATH248
course,,,,,,,,,,,,,,,,,,,,,
CS276,0.038613,0.037024,0.030470,0.021500,0.019509,0.018370,0.018101,0.016456,0.014715,0.013437,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CEE176B,0.006059,0.000000,0.016956,0.012128,0.000000,0.000000,0.000000,-0.003740,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MS&E252,0.016742,0.054770,0.051094,0.021722,0.066643,0.015836,0.000000,0.013272,0.017568,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EARTHSYS111,0.000000,0.000000,0.018820,0.080774,0.010652,0.000806,NaN,0.000000,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PSYCH131,-0.004587,0.005964,0.021081,0.005305,0.007391,-0.008478,0.000708,-0.004582,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHEMENG162,NaN,0.003456,0.004683,0.005080,0.003456,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ME140,NaN,0.009609,0.032296,0.015827,-0.002503,NaN,NaN,0.002054,0.000000,0.003095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HUMBIO128,NaN,NaN,0.009460,0.000000,0.001916,0.001954,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
# normalize by making all rows sum to 1
norm_highach_coefs_full = rowwise_normalize(highach_coefs_full)
display(norm_highach_coefs_full)

,CS161,CS109,MATH51,ECON1,CS103,PSYCH50,CS224U,CS221,CS145,CS148,...,PHYSICS364,COMM122,ME300B,MUSIC421A,COMM274D,GEOPHYS281,LINGUIST205A,PHIL180A,PHIL350A,MATH248
course,,,,,,,,,,,,,,,,,,,,,
CS276,0.137297,0.131646,0.108343,0.076447,0.069367,0.065319,0.064361,0.058513,0.052324,0.047778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CEE176B,0.012248,0.000000,0.034279,0.024518,0.000000,0.000000,0.000000,-0.007560,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MS&E252,0.018333,0.059976,0.055950,0.023787,0.072977,0.017341,0.000000,0.014533,0.019238,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EARTHSYS111,0.000000,0.000000,0.034721,0.149019,0.019651,0.001486,NaN,0.000000,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PSYCH131,-0.036646,0.047647,0.168412,0.042378,0.059045,-0.067730,0.005659,-0.036608,0.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CHEMENG162,NaN,0.014011,0.018987,0.020595,0.014011,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ME140,NaN,0.009182,0.030859,0.015122,-0.002391,NaN,NaN,0.001963,0.000000,0.002957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HUMBIO128,NaN,NaN,0.063443,0.000000,0.012847,0.013107,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
# the 50 most predictive courses in the dataset, in ranked order
norm_highach_coefs_full.sum().sort_values(ascending=False)[:50]

CS106B         13.208465
CS107          13.041753
MATH51         12.990425
CS103          12.886159
CME100          9.866666
CS109           8.810540
CHEM33          8.088419
MATH53          7.836511
CS106A          7.714381
PHYSICS43       6.384442
CS106X          5.841443
MATH52          5.705910
CHEM35          5.511242
CHEM31B         5.310957
ECON50          5.156150
CS161           5.082639
CHEM31X         4.995264
ME203           4.956580
CS110           4.812288
ENGR40M         4.762297
ENGR14          4.492100
ENGR30          4.140742
PHYSICS41       3.860779
PSYCH1          3.846731
CME102          3.726181
ME101           3.677626
ECON102A        3.557849
PHYSICS63       3.526665
CHEM31A         3.399451
CHEM131         3.306694
CME100A         3.265342
ECON1           3.136320
BIO41           3.130638
CS124           3.096313
CS229           3.094813
CHEM171         3.082893
PHYSICS61       3.001130
EE102A          2.980841
ENGR15          2.895953
ME112           2.744625
